# Deploy Zookeeper

In [1]:
import pprint, time, sys

In [2]:
varClouderaPath = "/vagrant/notebooks/cloudera"
sys.path.append(varClouderaPath)

varClouderaPath

'/vagrant/notebooks/cloudera'

In [3]:
sys.path

['',
 '/opt/anaconda/lib/python27.zip',
 '/opt/anaconda/lib/python2.7',
 '/opt/anaconda/lib/python2.7/plat-linux2',
 '/opt/anaconda/lib/python2.7/lib-tk',
 '/opt/anaconda/lib/python2.7/lib-old',
 '/opt/anaconda/lib/python2.7/lib-dynload',
 '/opt/anaconda/lib/python2.7/site-packages',
 '/opt/anaconda/lib/python2.7/site-packages/Sphinx-1.4.1-py2.7.egg',
 '/opt/anaconda/lib/python2.7/site-packages/setuptools-23.0.0-py2.7.egg',
 '/opt/anaconda/lib/python2.7/site-packages/IPython/extensions',
 '/root/.ipython',
 '/vagrant/notebooks/cloudera']

In [4]:
!free

             total       used       free     shared    buffers     cached
Mem:       3922528    2193800    1728728        164      55204    1688956
-/+ buffers/cache:     449640    3472888 
Swap:      1015804          0    1015804 


# Connect to Cloudera Manager

In [19]:
from cm_api.endpoints.services import ApiService
from cm_api.endpoints.services import ApiServiceSetupInfo
from cm_api.api_client import ApiResource

In [20]:
varFullHost = "pocd-cm581-dev-manager.poc-d.internal"

varAdminUser = "admin"
varAdminPassword = "admin"

varApiResource = ApiResource(varFullHost, version=10, username=varAdminUser, password=varAdminPassword)
varManager = varApiResource.get_cloudera_manager()
varManager

In [7]:
# Get cluster

In [21]:
varClusterName = "POC-D Cluster"


In [22]:
varCluster = varApiResource.get_cluster(varClusterName)


In [23]:
!mkdir -p /opt/cloudera/lib/zookeeper
!mkdir -p /opt/cloudera/data/zookeeper
!mkdir -p /opt/cloudera/log/zookeeper

!chmod -R 777 /opt/cloudera/lib/zookeeper
!chmod -R 777 /opt/cloudera/data/zookeeper
!chmod -R 777 /opt/cloudera/log/zookeeper

In [24]:
varZooKeeperServiceConfig = {
    'zookeeper_datadir_autocreate': 'true',
    'enable_config_alerts': "true",
    'process_groupname': "zookeeper",
    'process_username': "zookeeper",
}


In [25]:
varZooKeeperRoleConfig = {
    'quorumPort': 3181,
    'electionPort': 4181,
    'clientPort': 2181,
    'dataLogDir': "/opt/cloudera/lib/zookeeper",
    'dataDir': "/opt/cloudera/data/zookeeper",
    'maxClientCnxns': 1024,
    'zookeeper_server_java_heapsize': 268435456,
    'oom_heap_dump_dir': "/tmp",
    'oom_heap_dump_enabled': False,
    'zk_server_log_dir': "/opt/cloudera/log/zookeeper",
    'max_log_backup_index': 10,
    'max_log_size': 100,
    'log_threshold': "INFO",
    'enable_config_alerts': "true",
}


In [29]:
varZK_Name = "ZooKeeper"
varZK_Hosts = "pocd-cm581-dev-man1.poc-d.internal".split(',')
varZK_Hosts

['pocd-cm581-dev-man1.poc-d.internal']

In [34]:
varZOOKEEPER = varCluster.create_service(varZK_Name, "ZOOKEEPER")
varZOOKEEPER.update_config(varZooKeeperServiceConfig)
varZOOKEEPER

In [35]:
varZKID = 0
for zkhost in varZK_Hosts:
    varZKID += 1
    varZOOKEEPER.create_role(varZK_Name + "-" + str(varZKID), "SERVER", zkhost)


In [43]:
varTest = varZOOKEEPER.get_all_role_config_groups()
varTest[0].name

u'ZooKeeper-SERVER-BASE'

In [44]:
varZOOKEEPER.get_role_config_group("ZooKeeper-SERVER-BASE").update_config(varZooKeeperRoleConfig)

{u'clientPort': u'2181',
 u'dataDir': u'/opt/cloudera/data/zookeeper',
 u'dataLogDir': u'/opt/cloudera/lib/zookeeper',
 u'electionPort': u'4181',
 u'enable_config_alerts': u'true',
 u'log_threshold': u'INFO',
 u'maxClientCnxns': u'1024',
 u'max_log_backup_index': u'10',
 u'max_log_size': u'100',
 u'oom_heap_dump_dir': u'/tmp',
 u'oom_heap_dump_enabled': u'false',
 u'quorumPort': u'3181',
 u'zk_server_log_dir': u'/opt/cloudera/log/zookeeper',
 u'zookeeper_server_java_heapsize': u'268435456'}

In [45]:
varZOOKEEPER.init_zookeeper()

In [46]:
varCluster.start().wait()